In [3]:
pip install tweepy
#pip install tensorflow


SyntaxError: invalid syntax (170689579.py, line 1)

#Twiiter KEY
7PLFddvaCmKONCFjeR02NLlVI

#Twitter KEy Secret
YSi0ziD0QrnrCkmRLTI7WehkcCMI5bhTNsKD9xYAN5ICSg2j0H

In [ ]:
import tweepy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, Dense, Input, Dropout, Bidirectional, MultiHeadAttention, LayerNormalization, Flatten

import numpy as np

# Replace with your own Twitter API credentials
consumer_key = '7PLFddvaCmKONCFjeR02NLlVI'
consumer_secret = 'YSi0ziD0QrnrCkmRLTI7WehkcCMI5bhTNsKD9xYAN5ICSg2j0H'
access_token = '162264807-3X2yMXmNpe6bbe80YlxWAEK1yOj4ucub5tURHTAf'
access_token_secret = 'yV1asyDR8GVgcSxpm2Jt1r5PAU8V4RnsNX1UZgAfzPp3k'

# Authenticate with Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [ ]:
tweets = api.search(q='programming', count=100)  # Replace 'programming' with your search term
sentences = [tweet.text for tweet in tweets]
labels = [1 if 'love' in tweet.text else 0 for tweet in tweets]  # Simplified labeling, 1 for positive, 0 for negative


In [ ]:
tokenizer = Tokenizer(num_words=500, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=50, padding="post")

labels = np.array(labels)


In [ ]:
model_cnn = Sequential([
    Embedding(500, 16, input_length=50),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn.fit(padded_sequences, labels, epochs=5)


In [ ]:
model_rnn = Sequential([
    Embedding(500, 16, input_length=50),
    LSTM(128),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.fit(padded_sequences, labels, epochs=5)


In [ ]:
class TransformerBlock(Sequential):
    def __init__(self, embed_dim, num_heads):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ln1 = LayerNormalization(epsilon=1e-6)
        self.ln2 = LayerNormalization(epsilon=1e-6)
        self.dense = Dense(units=embed_dim, activation="relu")

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        out1 = self.ln1(inputs + attn_output)
        ffn_output = self.dense(out1)
        return self.ln2(out1 + ffn_output)

input_layer = Input(shape=(50,))
embedding_layer = Embedding(500, 64)(input_layer)
transformer_block = TransformerBlock(64, 4)(embedding_layer)
flatten = Flatten()(transformer_block)
output_layer = Dense(1, activation='sigmoid')(flatten)

model_transformer = Model(inputs=input_layer, outputs=output_layer)
model_transformer.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_transformer.fit(padded_sequences, labels, epochs=5)
